In [1]:
import unidecode
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from datetime import timedelta

In [2]:
from matplotlib import pyplot as plt

In [3]:
from configs import (lista_columnas_fijas, 
                     lista_nombres_nuevos, 
                     dict_nombres_nuevos, 
                     lista_columnas_necesarias, 
                     barrios)

## Funciones para crear tabla base

In [4]:
def crear_diccionario_nombres_nuevos(df):
    
    nombres_nuevos = {}
    
    lista_nombres_viejos = list(df.columns)
    for nombre_viejo in lista_nombres_viejos:
        
        # limpiamos columnas del df (lowercase, acentos y emojis)
        nombre_viejo_limpio = unidecode.unidecode(nombre_viejo.lower())
        
        # si tiene combo lo deja igual
        if ('combo' in nombre_viejo_limpio) or ('bolson' in nombre_viejo_limpio):
            for key, value in dict_nombres_nuevos.items():
                if key in nombre_viejo_limpio:
                    nombres_nuevos[nombre_viejo] = value
        else:
            for nombre_nuevo in lista_nombres_nuevos:
                # si encuentra un substring en nombres nuevos que matchee se lo asigna
                if nombre_nuevo in nombre_viejo_limpio and not 'budin' in nombre_viejo_limpio:
                    nombres_nuevos[nombre_viejo] = nombre_nuevo
            
    return nombres_nuevos 

In [5]:
def sabado_posterior(fecha):
    sabado = fecha
    while(sabado.weekday() != 5):
        sabado = sabado + timedelta(days = 1)
    return sabado

cortar_fecha = lambda fecha_hora: fecha_hora.split(' ')[0]

def agregar_fecha_entrega(df):
    df['marca temporal'] = df['marca temporal'].apply(cortar_fecha)
    df['marca temporal'] = pd.to_datetime(df['marca temporal'],format='%Y/%m/%d')
    df['fechas entrega'] = df['marca temporal'].apply(sabado_posterior)
    return df

In [6]:
def limpiar_barrios(retirar):
    
    for barrio in barrios:
        if barrio in str(retirar):
            return barrio
    
    if retirar == 0:
        return 'Domicilio'
    
    return retirar

In [7]:
def crear_tabla_base(df):
    # rellena nans con 0
    df_filtrado = df.copy().fillna(0)
    
    # cambia los nombres de las columnas
    df_filtrado.rename(columns=crear_diccionario_nombres_nuevos(df), inplace=True)

    # filtra columnas con las que trabajar
    df_filtrado = df_filtrado[lista_columnas_necesarias]
    
    # agrega fecha de entrega segun fecha de pedido
    df_filtrado = agregar_fecha_entrega(df_filtrado)
    
    # limpia columna retirar
    df_filtrado['retirar'] = df_filtrado['retirar'].apply(limpiar_barrios)
    return df_filtrado 

In [8]:
df_original = pd.read_csv('2022_anonimize.csv')

df = crear_tabla_base(df_original)
df.head()

,marca temporal,nombre de usuario,retirar,choclo,palta,tomate perita,morron verde,uva,berenjena agroecologica,ajo,ciruela,manzana roja,limon,pera,naranja,naranja,durazno,morron rojo,zapallito redondo,tomate cherry,mandarina,frutillas,banana,mango,arandanos,jengibre,bolson de verdes,bolson de pesadas,combo bolson verdes pesadas,bolson citricos,fechas entrega
0,2021-03-28,0,Palermo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03
1,2021-03-28,1,Palermo,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03
2,2021-03-28,2,Villa Urquiza,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03
3,2021-03-28,3,Palermo,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2021-04-03
4,2021-03-28,4,Palermo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03


# Logica de filtrado

In [9]:
def aplicar_filtros(df, fecha=None, fecha_inicial=None, fecha_final=None, productos=None, barrios=None):
    df_filtrado = df.copy()
    
    if fecha:
        if fecha_inicial and fecha_final:
            print('Esta mal usada la funcion de filtros (escribir solo fecha, o solo fecha inicial y fecha final)')
        else:
            fecha_inicial = fecha
            fecha_final = fecha
    if fecha_inicial and fecha_final:
        df_filtrado = df_filtrado[(df_filtrado['fechas entrega'] >= fecha_inicial) &
                                  (df_filtrado['fechas entrega'] <= fecha_final)]
    if productos:
        df_filtrado = df_filtrado[productos+lista_columnas_fijas]
    if barrios:
        df_filtrado = df_filtrado[df_filtrado['retirar'].isin(barrios)]
        
    return df_filtrado

In [10]:
# filtrar pedidos unicos
df_cant_pedidos = df[['nombre de usuario', 'fechas entrega']].groupby(by=['nombre de usuario']).count()
df_cant_pedidos = df_cant_pedidos.reset_index()
usuarios_pedido_unico = df_cant_pedidos[df_cant_pedidos['fechas entrega'] == 1]['nombre de usuario'].tolist()

df[df['nombre de usuario'].isin(usuarios_pedido_unico)]

,marca temporal,nombre de usuario,retirar,choclo,palta,tomate perita,morron verde,uva,berenjena agroecologica,ajo,ciruela,manzana roja,limon,pera,naranja,naranja,durazno,morron rojo,zapallito redondo,tomate cherry,mandarina,frutillas,banana,mango,arandanos,jengibre,bolson de verdes,bolson de pesadas,combo bolson verdes pesadas,bolson citricos,fechas entrega
5,2021-03-28,5,Villa Urquiza,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03
6,2021-03-28,6,Villa Urquiza,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03
9,2021-03-29,9,Villa Urquiza,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2021-04-03
17,2021-03-30,17,Villa Urquiza,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2021-04-03
21,2021-03-31,21,Palermo,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2021-04-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2022-03-03,362,Villa Urquiza,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-03-05
1023,2022-03-03,363,Villa Crespo,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-03-05
1026,2022-03-03,364,Domicilio,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2022-03-05
1028,2022-03-03,365,Villa Crespo,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-03-05


### Planificación de los gráficos para implementar:
#### Para una sola fecha de entrega:
1. Lista de productos, seccionada por lista de casas populares (gráfico de barras)
2. Cantidad de pedidos totales por casa popular (gráfico de barras)

#### Para un rango de fechas de entrega:
3. Un solo producto, pedidos seccionados por casas populares (gráfico de barras seccionadas) 
4. Cantidad de pedidos totales por casa popular (gráfico de barras seccionadas)
5. Lista de productos (gráfico de líneas)
6. Cantidad de pedidos únicos por casa popular (scatter de colores)

#### Pedidos únicos:
7. Cantidad de pedidos únicos totales por casa popular (gráfico de torta)

**Obs**: para todo filtro tenemos que darle al usuario un conjunto de parametros validos

**Obs**: en casos de envío a domicilio, hay más información en la columna _Barrio_ que no estamos usando por el momento


In [11]:
import plotly.express as px


In [12]:
## grafico 1

def graficar_productos_por_fecha_barrios(df, productos, fecha, barrios):
    df_filtrado = aplicar_filtros(df, fecha=fecha, productos=productos, barrios=barrios)
    df_agrupado = df_filtrado.groupby(by=['retirar']).sum().reset_index()
    df_agrupado = pd.melt(df_agrupado, id_vars=['retirar'], value_vars=productos)
    fig = px.bar(df_agrupado, x='variable', y='value', color='retirar', title=fecha, labels={
        'variable':'productos',
        'value':'cantidad'
    })
    fig.show()


In [13]:
## grafico 2

def graficar_pedidos_por_fecha(df, fecha):
    df_filtrado = aplicar_filtros(df, fecha)
    df_agrupado = df_filtrado.groupby(by=['retirar']).count().reset_index()
    df_agrupado = df_agrupado[['retirar','fechas entrega']]

    fig = px.pie(df_agrupado, values='fechas entrega', names='retirar', title=fecha, labels={
            'fechas entrega':'cantidad'
        })

    fig.show()

In [17]:
## grafico 3

def graficar_producto_por_fecha_inicial_final(df, producto, fecha_inicial, fecha_final):
    df_filtrado = aplicar_filtros(df, fecha_inicial=fecha_inicial, fecha_final=fecha_final, productos=[producto])

    df_agrupado = df_filtrado.groupby(by=['retirar', 'fechas entrega']).sum().reset_index()
    fig = px.bar(df_agrupado, x='fechas entrega', y=producto, color='retirar', title='{} - {}'.format(fecha_inicial, fecha_final))
    fig.show()


In [18]:
fecha = '2022-03-05'
productos = ['bolson de verdes','bolson de pesadas','combo bolson verdes pesadas','uva', 'berenjena agroecologica', 'ajo']
barrios = ['Palermo', 'Villa Urquiza', 'Villa Crespo']

graficar_productos_por_fecha_barrios(df, productos, fecha, barrios)

In [19]:
fecha = '2022-03-05'

graficar_pedidos_por_fecha(df, fecha)


In [20]:
fecha_inicial = '2021-04-03'
fecha_final = '2021-08-07'
producto = 'bolson de verdes'

graficar_producto_por_fecha_inicial_final(df, producto, fecha_inicial, fecha_final)